# Spectrum of Linear Response Functions

In this report, I study the properties of the linear fit obtained by MCA regression of the temporally differenced data. As usual $X$ is the data matrix formed by concatenating the vertical profiles the $q_T$ and $s_l$ for each spatial location, $X'$ is the version of this data matrix shifted forward by one time point, and $F$ is the matrix of known source terms (not shifted in time). Also, $W_X$ is a diagonal matrix which weights each vertical location by the square root of its current layers mass and normalizes the different physical variables. The scale of the each variable is defined as the square root of the mass-weighted vertical average of the variance. In other words,
$$
W_X[i+n_z \alpha, i+n_z \alpha] = \sqrt{\frac{\Delta p_i}{\sigma^2_\alpha}},
$$
where
$$
\sigma_{\alpha}^2 = \sum_{i}^{n_z} \text{Var}_{x,y,t}(\alpha) \Delta p_i.
$$


Then, the time derivative is approximated by
$$ 
\frac{X-X'}{\Delta t}.
$$

In otherwords, if the loading matrices $\Phi_X$ and $\Phi_Y$ are given by $\Phi_X, \Phi_Y \leftarrow MCA(W_X( X'-X), W_X( X+\Delta t F);n=4)$. Once the MCA loadings are obtained, they are used to compute the MCA scores of the inputs, which are then regressed against the desired outputs $Y=X'-X - \Delta t F$. This whole procedure specifies the linear model
$$
X'-(X + \Delta t F) = M \Phi_X^T W_X (X+ \Delta t F) + E.
$$
This model can be rewritten as $X' = (I+M \Phi_X^T W_X) (X+ \Delta t F) + E$. Then, the linear generator of this process is approximated by $A = \frac{\log(I+M \Phi_X^T W_X)}{\Delta t}$.

The variable $\tilde{X}=X+ \Delta t F$ has the interpretation of the variable after all known source terms have updated the solution state. This can be interprested as the model state at the end of a climate model's time step. Therefore, predicting $X'$ from $\tilde{X}$ amounts to predicting the beginning of the next time from the end of the current time step. This formulation allows an easy implementation inside a complicated climate model by monitoring the model state at the end of each time step.

In [ ]:
%matplotlib inline

In [ ]:
from sklearn.externals import joblib
from sklearn.linear_model import LinearRegression
import xarray as xr
import pandas as pd
import numpy as np


import holoviews as hv
hv.extension('bokeh')

from lib.lrf import plot_lrf
from lib.mca import MCARegression

Load the DMD data in.

In [ ]:
p = xr.open_dataset("../data/raw/ngaqua/stat.nc").p
dmd_data = joblib.load("../data/ml/dmd.pkl")

x_train, y_train = dmd_data['train']
scale_in, scale_out = dmd_data['scale']
weight_in, weight_out = dmd_data['weight']

In [ ]:
# np.linalg.cond(x_train)

It appears to be infinity, so some sort of regularization is needed. Uncommnent the code cell above if you want to see for your self.

# Principal components analysis

In [ ]:
from sklearn.decomposition import PCA
ntrain = 400000
idx = np.random.choice(x_train.shape[0], ntrain)

What is the PCA structure like?

In [ ]:
pca = PCA(10).fit(x_train[idx]*np.sqrt(weight_in)/scale_in)

In [ ]:
hv.Bars(pca.explained_variance_ratio_)

The first mode is fairly dominant, but the variance explained does not fall off very quickly.

# MCA Regression

We use four MCA components (the default value in our MCARegression code).

In [ ]:
# MCA
mca = MCARegression(LinearRegression(),
                    n_components=5,
                    scale=(np.sqrt(weight_in)/scale_in, np.sqrt(weight_out)/scale_out))

mca.fit(x_train, y_train-x_train)

Now, we extract the linear operator $B= I + M \Phi_X W_X$.

In [ ]:
# need to add identity again
I = np.eye(x_train.shape[1])
B  = (mca.predict(I) - mca.predict(0*I)) +I

hv.Image(B).to.curve("x")

These functions help plot the spectra of $B$.

In [ ]:
def sorted_eig(A):

    lam, P = np.linalg.eig(A)
    lam_sort = np.abs(lam).argsort()
   
    lam = lam[lam_sort[::-1]]
    P = P[:,lam_sort[::-1]]
    
    return lam, P

def eigvals_plot(lam):
    return hv.Points((lam.real, lam.imag)).redim(x="Re", y="Im") * hv.Ellipse(0, 0, 2)

Here are the eigenvalues of $B$ plotted in the complex plane

In [ ]:
%%opts Points(size=4)
lam, P = sorted_eig(B)
eigvals_plot(lam)

The eigenvalues of this matrix are all within the unit circle, which indicates that the linear fit is stable. Now, we can examine the infinitesimal generator $A$.

In [ ]:
from scipy.linalg import logm, expm
A = logm(B)/(3/24)

I need to figure out why these results are different from the ones obtained in notebook `3.4-ndb-spectra-of-modes.ipynb`. In theory these matrices should be exactly the same.

In [ ]:
%%opts Curve {+axiswise}
hv.Image(P.real).to.curve("x") + hv.Image(P.imag).to.curve("x")

These eigenvalues are junk! I am not even sure what the DMD should look like for this problem.

# Plot LRF

Here is the plot of the generator $A$.

In [ ]:
# add important index information to the linear response function
lrf = pd.DataFrame(A, index=x_train.indexes['features'],
                   columns=y_train.indexes['features'])

In [ ]:
plot_lrf(lrf, p);

# Summary

- Many principal components are needed to compress the input data.
- The linear response function obtained by MCA Regression with 4 components is stable
- The eigenmodes of the $A$ are unintelligible.


I think the best path forward is to use the optimized DMD algorithm which fits for the best modes and eigenvalues simultaneously. Our current approach of finding the MCA modes, fitting a linear response matrix, and then taking its matrix logarithm can introduce significant bias. The oDMD is a state of the art method for performing all these steps simulateneously.